# Installing Packages and Dependencies

In [57]:
import csv
import tensorflow as tf
import os
import cv2
import numpy as np
from collections import deque
import datetime as dt
from moviepy import VideoFileClip

CLASSES_LIST = ['BaseballPitch', 'Basketball', 'BenchPress', 'Biking', 'Billiards', 'BreastStroke', 'CleanAndJerk', 'Diving', 'Drumming', 'Fencing', 'GolfSwing', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'JavelinThrow', 'JugglingBalls', 'JumpingJack', 'JumpRope', 'Kayaking', 'Lunges', 'MilitaryParade', 'Mixing', 'Nunchucks', 'PizzaTossing', 'PlayingGuitar', 'PlayingPiano', 'PlayingTabla', 'PlayingViolin', 'PoleVault', 'PommelHorse', 'PullUps', 'Punch', 'PushUps', 'RockClimbingIndoor', 'RopeClimbing', 'Rowing', 'SalsaSpin', 'SkateBoarding', 'Skiing', 'Skijet', 'SoccerJuggling', 'Swing', 'TaiChi', 'TennisSwing', 'ThrowDiscus', 'TrampolineJumping', 'VolleyballSpiking', 'WalkingWithDog', 'YoYo']
SEQUENCE_LENGTH = 20
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Loading Model

In [58]:
model = tf.keras.models.load_model(filepath='LRCN_model___Date_Time_2025_01_18__23_34_25___Loss_1.4214158058166504___Accuracy_0.7007779479026794.h5')
input_video_filepath = "Test_Rowing.mp4"
output_video_filepath = f"test_output/{input_video_filepath.replace(".mp4","")}_video-output.mp4"
result_csv_filepath = f"{input_video_filepath.replace(".mp4","")}_csv-output.csv"

# Main Implementation

### Writer/Appender Module

In [65]:
def append_on_csv(filename, dataline, mode="a"):
    with open(filename, mode,newline='\n') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(dataline)

### Prediction

In [66]:
def predict_on_video(video_file_path, output_file_path, SEQUENCE_LENGTH):
    current_date_time_start = dt.datetime.now()
    '''
    This function will perform action recognition on a video using the LRCN model.
    Args:
    video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
    output_file_path: The path where the ouput video with the predicted action being performed overlayed will be stored.
    SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
    '''

    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize the VideoWriter Object to store the output video in the disk.
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc(*'mp4v'), 
                               video_reader.get(cv2.CAP_PROP_FPS), 
                               (original_video_width, original_video_height))

    # Declare a queue to store video frames.
    frames_queue = deque(maxlen = SEQUENCE_LENGTH)

    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''

    # Iterate until the video is accessed successfully.
    while video_reader.isOpened():

        # Read the frame.
        ok, frame = video_reader.read() 
        
        # Check if frame is not read properly then break the loop.
        if not ok:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255
        confidence = 0
        # Appending the pre-processed frame into the frames list.
        frames_queue.append(normalized_frame)

        # Check if the number of frames in the queue are equal to the fixed sequence length.
        if len(frames_queue) == SEQUENCE_LENGTH:

            # Pass the normalized frames to the model and get the predicted probabilities.
            predicted_labels_probabilities = model.predict(np.expand_dims(frames_queue, axis = 0))[0]

            # Get the index of class with highest probability.
            predicted_label = np.argmax(predicted_labels_probabilities)
            confidence = predicted_labels_probabilities[predicted_label]
            # Get the class name using the retrieved index.
            predicted_class_name = CLASSES_LIST[predicted_label]
            current_date_time_iter = dt.datetime.now()
            delta = current_date_time_iter - current_date_time_start
            append_on_csv(result_csv_filepath,[100*confidence,predicted_class_name,f"{int(delta.total_seconds()//60)}hr {delta.total_seconds()%60}s"]) #will use current_date_time_string cuz i can't figure it out
            print(f"{100*confidence}% chance of being {predicted_class_name} {int(delta.total_seconds()//60)}hr {delta.total_seconds()%60}s") #will use current_date_time_string cuz i can't figure it out

        # Write predicted class name on top of the frame.
        cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Write The frame into the disk using the VideoWriter Object.
        video_writer.write(frame)
        
    # Release the VideoCapture and VideoWriter objects.
    video_reader.release()
    video_writer.release()

In [67]:

predict_on_video(input_video_filepath, output_video_filepath, SEQUENCE_LENGTH)
VideoFileClip(output_video_filepath, audio=False, target_resolution=(300,None)).preview()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
59.8602294921875 chance of being Rowing 0hr 0.073769s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
47.15468883514404 chance of being Basketball 0hr 0.12577s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
74.97779726982117 chance of being Rowing 0hr 0.176538s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
87.08177208900452 chance of being Rowing 0hr 0.227887s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
90.0201141834259 chance of being Rowing 0hr 0.281132s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
86.69770359992981 chance of being Rowing 0hr 0.335011s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
82.02697038650513 chance of being Rowing 0hr 0.387796s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
77.17272639274597 chance of being Rowing 0hr 0.445175s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
71.60723209381104 chance of being Rowing 0hr 0.498586s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
64.23686146736145 chance of being Rowing 0hr 0.548588s
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
60.487014055252075 chance of be

OSError: [Errno 32] Broken pipe

MoviePy error: FFPLAY encountered the following error while previewing clip :

 None